In [0]:
import requests
import pandas as pd
import numpy as np
from pandas import read_csv
import glob
import time
import datetime
import os
import os.path

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
# Get tickers
ticker = read_csv('/content/gdrive/My Drive/Data/TICKER.csv',header = None)
path = '/content/gdrive/My Drive/Data/Raw Data/'
IDList = list(Bank[0])
IDList.sort()

In [0]:
# Set Dates
start = '01/01/2015'
end = str(int(time.time()))
stamp = datetime.datetime.strptime(start, "%d/%m/%Y").timestamp()
stamp = str(int(stamp))

In [0]:
# Download Data
for i in IDList:
  site = 'https://query1.finance.yahoo.com/v7/finance/download/' + i + '?period1='+ stamp + '&period2=' + end + '&interval=1d&events=history&crumb=hP2rOschxO0'
  ##wk; mo; d
  response = requests.post(site)
  idint = i.replace('.hk','')
  idint = idint.replace('.HK','')
  save = path + idint + '.csv'
  dirname = os.path.dirname(save)
  if not os.path.exists(dirname):
    os.makedirs(dirname)
  with open(save , 'w') as f:
    f.writelines(response.text)
    print(i)

In [0]:
# Format the data
path = '/content/gdrive/My Drive/Data/'
filenames = glob.glob(path + 'Raw Data/' + "/*.csv")

Files = []
for name in filenames:
  df = read_csv(name, header=0,index_col=None)
  df=df.sort_values(by=['Date'])
  col = name.replace(path+ 'Raw Data/','')
  col = col.replace('.csv', '')
  df['ID'] = col
  Files = Files + [df]
  
feature = 'Close'

Output = pd.DataFrame({})
for file in Files:
  col = file['ID'].iloc[0]
  file = file.set_index('Date')
  temp=file[feature].to_frame()
  temp.columns=[col]
  if temp.index.duplicated().any():
    temp = temp.reset_index().drop_duplicates(subset='Date', keep='first').set_index('Date')
  Output=pd.concat([Output, temp], axis=1)
save = path+'WorkFile/'+feature + '.csv'
Output = Output.round(2)
Output.to_csv(save,index=True,na_rep='NA')
    
